In [ ]:
# %%
import sys, os

try:
    # ✅ Running from a Python script (.py file)
    base_path = os.path.abspath(os.path.join(os.path.dirname(__file__), ".."))
except NameError:
    # ✅ Running from a Jupyter notebook (__file__ is not defined)
    base_path = os.path.abspath(os.path.join(os.getcwd(), ".."))

SRC_PATH = os.path.join(base_path)

if SRC_PATH not in sys.path:
    sys.path.insert(0, SRC_PATH)
    print(f"✅ SRC path added: {SRC_PATH}")
else:
    print(f"🔁 SRC path already in sys.path: {SRC_PATH}")

In [ ]:
from langgraph.types import Command
from utils.intent_keywords import INTENT_KEYWORDS

def detect_intent(text: str) -> str:
    text = text.lower()
    for intent, keywords in INTENT_KEYWORDS.items():
        if any(kw in text for kw in keywords):
            return intent
    return "unknown"

def intent_node(state) -> Command:
    user_msg = [m for m in state["messages"] if m["role"] == "user"][-1]["content"]
    detected_intent = detect_intent(user_msg)

    return Command(
        update={
            "intent": detected_intent,
            "messages": state["messages"] + [{"role": "intent", "content": detected_intent}],
            "agent_name": "intent_router"
        },
        goto="supervisor"
    )
